In [1]:
import polars as pl
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

TRAIN_DATA_FILE_PATH = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
VAL_DATA_FILE_PATH = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

## Question 1 - Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* **19**

In [2]:
jan_df = pl.read_parquet(source=TRAIN_DATA_FILE_PATH)

In [3]:
len(jan_df.columns)

19

There is a total of 19 columns in the January 2023 dataset.

## Question 2 - Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

* 32.59
* **42.59**
* 52.59
* 62.59

In [4]:
jan_df = jan_df.with_columns(
    (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"))
    .dt.total_minutes()
    .alias("duration_minutes")
)

In [5]:
jan_df['duration_minutes'].std()

42.59449915808725

Standard deviation for the `duration_minutes` variable is 42.59.

## Question 3 - Dropping Outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep
only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records is left after you dropped the outliers?

* 90%
* 92%
* 95%
* **98%**

In [6]:
new_jan_df = jan_df.filter(
    (pl.col("duration_minutes") >= 1) &
    (pl.col("duration_minutes") <= 60)
)

In [7]:
len(new_jan_df) / len(jan_df)

0.9818303711466737

There is 98% of the original data left after dropping outliers.

## Question 4 - One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will
  label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [8]:
feature_vars = ['PULocationID', 'DOLocationID']
train_df = new_jan_df.select(feature_vars).with_columns(
    pl.col(feature_vars).cast(pl.String)
)

In [9]:
dv = DictVectorizer(sparse=False)

train_dicts = train_df.to_pandas().to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [10]:
X_train.shape

(3011044, 515)

There is 515 total number of columns for `X_train`.

## Question 5 - Training a model

Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [11]:
target_var = 'duration_minutes'
train_vector = new_jan_df[target_var]
y_train = train_vector.to_numpy()

In [ ]:
lr = LinearRegression(n_jobs=8)
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

## Question 6 - Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [ ]:
feb_df = pl.read_parquet(source=VAL_DATA_FILE_PATH)

feb_df = feb_df.with_columns(
    (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"))
    .dt.total_minutes()
    .alias("duration_minutes")
)

In [ ]:
val_df = feb_df.select(feature_vars).with_columns(
    pl.col(feature_vars).cast(pl.String)
)

val_dicts = val_df.to_pandas().to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
y_val = val_df[target_var].to_numpy()

In [ ]:
y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)